## Vit
---

In [2]:
#![feature(portable_simd)]
use std::simd::{u64x8, u64x64, SimdUint};

In [3]:
#[derive(Debug, Default)]
struct Vit {
    vitlets: Vec<u64x64>,
    size: usize,
}

In [4]:
#![feature(slice_group_by)]
/// Vit, a bit vector based on portable simd primatives.
impl Vit {
    fn new(size: usize) -> Self {
        let num_vitlets = (size as f64 / 4096f64).ceil() as usize;
        let vitlets = [u64x64::splat(0)].into_iter().cycle().take(num_vitlets).collect();
        Vit {
            vitlets: vitlets,
            size: size,
        }
    }
    fn offsets(offset: usize) -> (usize, usize, usize) {
        let offset1 = offset / 4096;
        let offset2 = (offset - 4096 * offset1) / 64;
        let offset3 = offset % 64;
        (offset1, offset2, offset3)
    }
    fn set(&mut self, offset: usize, val: bool) {
        let (offset1, offset2, offset3) = Vit::offsets(offset);
        let mut temp = self.vitlets[offset1][offset2];
        let is_set = (temp & (1 << offset3)) != 0;
        if (is_set & !val) | (!is_set & val) {
            temp = temp ^ (1 << offset3);
        }
        self.vitlets[offset1][offset2] = temp;
    }
    fn check(&self, offset: usize) -> bool {
        let (offset1, offset2, offset3) = Vit::offsets(offset);
        let temp = self.vitlets[offset1][offset2];
        (temp & (1 << offset3)) != 0
    }
    fn check_multi(&self, offsets: &[usize]) -> bool {
        let mut result = true;
        for offset in offsets.iter() {
            let (offset1, offset2, offset3) = Vit::offsets(*offset);
            let temp = self.vitlets[offset1][offset2];
            result &= (temp & (1 << offset3)) != 0;
            if !result {
                return false;
            }
        }
        true
    }
    fn check_multi_v2(&self, offsets: &[usize]) -> bool {
        let mut offsets = offsets.to_owned();
        offsets.sort();
        let offsets = offsets.iter().map(|x|Vit::offsets(*x)).collect::<Vec<_>>().to_owned();
        let offsets = offsets.group_by(|x,y|(x.0,x.1)==(y.0,y.1)).collect::<Vec<_>>();
        for simd_grp in offsets.iter() {
            let mut simd_mask = u64x64::splat(0);
            let offset1 = simd_grp[0].0;
            let offset2 = simd_grp[0].1;
            for off_tup in simd_grp.iter() {
                simd_mask[offset2] ^= 1 << off_tup.2;
            }
            let simd_check = self.vitlets[offset1] & simd_mask == simd_mask;
            if !simd_check {
                return false;
            }
        }
        true
    }
}

In [5]:
let mut v = Vit::new(5000);
v

Vit { vitlets: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], size: 5000 }

In [6]:
v.set(0, true);
v.set(1, true);
v.set(2, true);
v.set(64, true);
v.set(64*2, true);
v.set(64*3, true);
v

Vit { vitlets: [[7, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], size: 5000 }

In [7]:
([v.check(0), v.check(2)].iter().fold(true, |a,b|a&b), [v.check(2), v.check(20)].iter().fold(true, |a,b|a&b))

(true, false)

In [8]:
(v.check_multi(&[0, 2]), v.check_multi(&[2, 20]))

(true, false)

In [9]:
(v.check_multi_v2(&[0, 2]), v.check_multi_v2(&[2, 20]))

(true, false)

---
## No clue how to use __m512i:

In [10]:
#![feature(stdsimd)]
use std::arch::x86_64::{__m512i, _mm512_and_epi64};

In [11]:
let a = __m512i::from(u64x8::from([0u64, 1, 1, 2, 2, 3, 3, 4]));
let b = __m512i::from(u64x8::from([0u64, 1, 2, 3, 4, 5, 6, 7]));
(a, b)

(__m512i(0, 1, 1, 2, 2, 3, 3, 4), __m512i(0, 1, 2, 3, 4, 5, 6, 7))

In [12]:
// Jupyter may not be down with this?
#[target_feature(enable = "avx512f")]
unsafe fn doit(a: __m512i, b: __m512i) -> __m512i {
    unsafe { _mm512_and_epi64(a, b) }
}

Error: the target feature `avx512f` is currently unstable